# Layout

+ Intro algebra
    + news
    + physics
+ doc2vec
    + press releases
+ score function
    + gensim
+ dimensions example, e.g. he-she

In [1]:
#All these packages need to be installed from pip
import gensim#For word2vec, etc
import requests #For downloading our datasets
import nltk #For stop words and stemmers
import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import random

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning
%matplotlib inline

import os #For looking through files
import os.path #For managing file paths


# Intro

intro stuff ...

# Getting our corpuses

Instead of downloading our corpora, we have download them ahead of time, a subset of the [senate press releases](https://github.com/lintool/GrimmerSenatePressReleases) are in `data/grimmerPressReleases`. So we will load them into a DataFrame, to do this first we need to define a function to convert directories of text files into DataFrames.

In [2]:
def loadDir(targetDir, category):
    allFileNames = os.listdir(targetDir)
    #We need to make them into useable paths and filter out hidden files
    filePaths = [os.path.join(targetDir, fname) for fname in allFileNames if fname[0] != '.']

    #The dict that will become the DataFrame
    senDict = {
        'category' : [category] * len(filePaths),
        'filePath' : [],
        'text' : [],
    }

    for fPath in filePaths:
        with open(fPath) as f:
            senDict['text'].append(f.read())
            senDict['filePath'].append(fPath)

    return pandas.DataFrame(senDict)

Now we can use the function in all the directories in `data/grimmerPressReleases`

In [3]:
dataDir = 'data/grimmerPressReleases'

senReleasesDF = pandas.DataFrame()

for senatorName in [d for d in os.listdir(dataDir) if d[0] != '.']:
    senPath = os.path.join(dataDir, senatorName)
    senReleasesDF = senReleasesDF.append(loadDir(senPath, senatorName), ignore_index = True)

senReleasesDF[:100:10]

,category,filePath,text
0,Kennedy,data/grimmerPressReleases/Kennedy/01Apr2005Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE...
10,Kennedy,data/grimmerPressReleases/Kennedy/01Dec2005Ken...,FOR IMMEDIATE RELEASE Washington ...
20,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2006Ken...,FOR IMMEDIATE RELEASE Fact sheet...
30,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2007Ken...,FOR IMMEDIATE RELEASE Washington ...
40,Kennedy,data/grimmerPressReleases/Kennedy/01Jun2007Ken...,FOR IMMEDIATE RELEASE BOSTON MA Se...
50,Kennedy,data/grimmerPressReleases/Kennedy/01Mar2007Ken...,FOR IMMEDIATE RELEASE Washington ...
60,Kennedy,data/grimmerPressReleases/Kennedy/01May2007Ken...,FOR IMMEDIATE RELEASE The President ...
70,Kennedy,data/grimmerPressReleases/Kennedy/01Nov2007Ken...,FOR IMMEDIATE RELEASE Washington DC...
80,Kennedy,data/grimmerPressReleases/Kennedy/02Aug2006Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE ...
90,Kennedy,data/grimmerPressReleases/Kennedy/02Feb2005Ken...,FOR IMMEDIATE RELEASE The Preside...


# Stemming is taking a really long time do to the size of the dataset, so it's been disabled for now

We also want to remove stop words and stem, but tokenizing requires two steps. Word2Vec wants to know the sentence structure as well as simply the words, so the tokenizing is slightly different this time.

In [4]:
#Define the same function as last week
def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None, lemmer = None):
    #We can use a generator here as we just need to iterate over it

    #Lowering the case and removing non-words
    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    #Now we can use the semmer, if provided
    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)

    #And the lemmer
    if lemmer is not None:
        workingIter = (lemmer.lemmatize(w) for w in workingIter)

    #And remove the stopwords
    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)
    #We will return a list with the stopwords removed
    return list(workingIter)

#initialize our stemmer and our stop words
stop_words_nltk = nltk.corpus.stopwords.words('english')
snowball = nltk.stem.snowball.SnowballStemmer('english')
wordnet = nltk.stem.WordNetLemmatizer()

In [5]:
#Apply our functions, notice each row is a list of lists now
senReleasesDF['tokenized_sents'] = senReleasesDF['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
senReleasesDF['normalized_sents'] = senReleasesDF['tokenized_sents'].apply(lambda x: [normlizeTokens(s, stopwordLst = stop_words_nltk, stemmer = None) for s in x])

senReleasesDF[:100:10]

,category,filePath,text,tokenized_sents,normalized_sents
0,Kennedy,data/grimmerPressReleases/Kennedy/01Apr2005Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE...,"[[FOR, IMMEDIATE, RELEASE, FOR, IMMEDIATE, REL...","[[immediate, release, immediate, release, cont..."
10,Kennedy,data/grimmerPressReleases/Kennedy/01Dec2005Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, D, C, T...","[[immediate, release, washington, c, today, se..."
20,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2006Ken...,FOR IMMEDIATE RELEASE Fact sheet...,"[[FOR, IMMEDIATE, RELEASE, Fact, sheets, on, B...","[[immediate, release, fact, sheets, bush, plan..."
30,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2007Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, D, C, T...","[[immediate, release, washington, c, today, u,..."
40,Kennedy,data/grimmerPressReleases/Kennedy/01Jun2007Ken...,FOR IMMEDIATE RELEASE BOSTON MA Se...,"[[FOR, IMMEDIATE, RELEASE, BOSTON, MA, Senator...","[[immediate, release, boston, senator, edward,..."
50,Kennedy,data/grimmerPressReleases/Kennedy/01Mar2007Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, DC, Tod...","[[immediate, release, washington, dc, today, s..."
60,Kennedy,data/grimmerPressReleases/Kennedy/01May2007Ken...,FOR IMMEDIATE RELEASE The President ...,"[[FOR, IMMEDIATE, RELEASE, The, President, is,...","[[immediate, release, president, wrong, veto, ..."
70,Kennedy,data/grimmerPressReleases/Kennedy/01Nov2007Ken...,FOR IMMEDIATE RELEASE Washington DC...,"[[FOR, IMMEDIATE, RELEASE, Washington, DC, Sen...","[[immediate, release, washington, dc, senators..."
80,Kennedy,data/grimmerPressReleases/Kennedy/02Aug2006Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE ...,"[[FOR, IMMEDIATE, RELEASE, FOR, IMMEDIATE, REL...","[[immediate, release, immediate, release, impo..."
90,Kennedy,data/grimmerPressReleases/Kennedy/02Feb2005Ken...,FOR IMMEDIATE RELEASE The Preside...,"[[FOR, IMMEDIATE, RELEASE, The, President, gav...","[[immediate, release, president, gave, effecti..."


# Word2Vec

We will be using the gensim implementation of [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec).

To load our data our data we give all the sentences to the trainer

In [6]:
senReleasesW2V = gensim.models.word2vec.Word2Vec(senReleasesDF['normalized_sents'].sum())

Now we can look at a few things

In [7]:
senReleasesW2V.most_similar('president')

[('administration', 0.7631863951683044),
 ('presidents', 0.7629678249359131),
 ('administrations', 0.665790319442749),
 ('cheney', 0.6004915833473206),
 ('george', 0.5805345773696899),
 ('quoting', 0.5552192330360413),
 ('ronald', 0.528830885887146),
 ('responds', 0.523741602897644),
 ('rollback', 0.5176122188568115),
 ('veto', 0.5079611539840698)]

Get the vector

In [8]:
senReleasesW2V['president'] + senReleasesW2V['boston']

array([ 1.41920853, -0.29156959, -1.20161223, -1.21448827,  0.66914237,
        1.5699327 ,  2.80649996,  0.12193584,  1.71496856, -0.48102066,
        2.10617161, -0.62677407,  2.17801404, -1.83463788, -0.84686506,
       -0.22008485, -3.17664123,  0.1544082 , -1.58886182, -2.17827368,
        0.25125426, -2.27298927, -1.80616939,  2.04321122,  0.85683382,
       -0.07358402,  1.50461626,  0.59983718, -2.62412333,  1.25119257,
        0.39815867, -2.18994045,  0.1814287 , -0.79145753,  2.17259955,
       -1.29216838,  1.46731448,  2.40921164,  0.23850721,  1.72597623,
        1.4124999 , -3.36318588,  2.61210942, -2.47834206,  2.96945786,
        0.11076057,  2.69861627, -1.73596001, -0.86435795, -0.32480884,
        0.80850911,  1.48252547,  0.85056043, -0.30731586, -1.97084653,
       -2.48201585,  1.3761183 , -0.73786914,  2.87536144,  4.64721107,
        0.1104061 ,  2.91338253, -3.71677494, -2.85429859, -1.25716209,
       -2.6419673 , -0.83872062, -1.19794869,  1.60902977,  1.39

Get all the vectors

In [9]:
senReleasesW2V.syn0

array([[  7.35480636e-02,  -7.02913761e-01,   1.42821920e+00, ...,
          1.47880507e+00,   6.47500694e-01,  -6.15879774e-01],
       [  5.64537287e-01,   2.34248805e+00,   1.57239735e+00, ...,
         -2.91723281e-01,  -3.87601882e-01,  -3.40682387e-01],
       [ -2.66966891e+00,   1.38446510e+00,   1.72242844e+00, ...,
          4.02330256e+00,   9.52823758e-01,  -1.76577640e+00],
       ..., 
       [  7.86399320e-02,  -4.06351089e-02,   2.98865344e-02, ...,
          2.89073382e-02,   4.29980978e-02,   2.40515191e-02],
       [  8.22230726e-02,  -6.66982606e-02,  -3.50666642e-02, ...,
          7.92473480e-02,   8.96688271e-03,  -5.23903174e-03],
       [  3.72739369e-03,  -3.74005199e-03,   4.08855602e-02, ...,
          2.88671385e-02,   4.21586558e-02,   3.71375517e-03]], dtype=float32)

Find what doesn't fit

In [10]:
senReleasesW2V.doesnt_match(['administration', 'administrations', 'presidents', 'president', 'washington'])

'washington'

In [11]:
def myDoesntmatch(w2v, words):
    mean = w2v[words[0]]
    for w in words[1:]:
        mean += w2v[w]
    mean = mean / len(words)
    worst = words[0]
    wv = 0
    for w in words:
        v = abs(np.dot(mean, w2v[w]))
        if v > wv:
            wv = v
            worst = w
    return worst
myDoesntmatch(senReleasesW2V, ['administration', 'administrations', 'presidents', 'president', 'washington'])

'administration'

Or save for use later

In [12]:
senReleasesW2V.save("data/senpressreleasesWORD2Vec")

We can also use the documensts themselves instead of the sentences to generate the model, this is called [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html).

In [13]:
taggedDocs = []
for index, row in senReleasesDF.iterrows():
    #taggedDocs.append(row)
    taggedDocs.append(gensim.models.doc2vec.TaggedDocument(row['normalized_sents'], row['category']))
senReleasesDF['TaggedDocuments'] = taggedDocs

We can then train the model the same way as before

In [14]:
#senReleasesW2V = gensim.models.doc2vec.Doc2Vec(documents = list(senReleasesDF['TaggedDocuments']))

# APS abstracts

In [15]:
apsDF = pandas.read_csv('data/APSabstracts1950s.csv', index_col = 0)
apsDF['tokenized_sents'] = apsDF['abstract'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
apsDF['normalized_sents'] = apsDF['tokenized_sents'].apply(lambda x: [normlizeTokens(s, stopwordLst = stop_words_nltk, lemmer = wordnet) for s in x])

Here, let's change some parameters. Let's use skipgrams instead of CBOW (continous bag of words), have a 200-dimensional vector space, only keep words that appear more than 2 times, and iterate the training algorithm over the corpus 10 times. For more information about parametrizing word2vec models, please see [here](https://radimrehurek.com/gensim/models/word2vec.html) and [here](https://code.google.com/archive/p/word2vec/).

In [16]:
apsW2V = gensim.models.word2vec.Word2Vec(apsDF['normalized_sents'].sum(), sg = 1, size = 200, min_count= 2, iter=10)

Let's save the model.

In [17]:
apsW2V.save('data/apsW2V')

We can then also load it.

In [18]:
#apsW2V = gensim.models.word2vec.Word2Vec.load('data/random230K.model')#, binary=True)

Now let's do some vector algebra.

In [19]:
apsW2V.most_similar(positive = ['newton', 'relativity'], negative = ['einstein'], topn = 5)

[('ought', 0.7602584362030029),
 ('admitting', 0.7561383247375488),
 ('passive', 0.7540632486343384),
 ('speck', 0.7498645782470703),
 ('admissible', 0.7421443462371826)]

# News from The New York Times 

In [20]:
paragraphs = []
f = open('data/nytimes_full.txt', 'r')
for row in f:
    if row != '\n' and  row != "';\n":
        paragraphs.append(row)
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/nytimes_full.txt'

In [ ]:
indices = random.sample(range(len(paragraphs)), 5000)

In [ ]:
sample = [paragraphs[i] for i in sorted(indices)]